# **Installation**

In [ ]:
pip install mediapipe --user

# **Imports**

In [ ]:
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
path = 'Data/Video/input2.MOV'
video = cv2.VideoCapture(path)
count = 0
frames_arr = []
while video.isOpened():
    ret, frame = video.read()

    if not ret:
        break
    frames_arr.append(frame)                                         # Frames stored in frames_arr
    count += 1
video.release()
print(len(frames_arr), count)


img = frames_arr[0]
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
frames = []
frames.append(frames_arr[0])
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
for i in range(1, len(frames_arr)):

    keypoints1, descriptors1 = orb.detectAndCompute(frames[-1], None)
    keypoints2, descriptors2 = orb.detectAndCompute(frames_arr[i], None)

    matches = bf.match(descriptors1, descriptors2)

    #matches = sorted(matches, key=lambda x: x.distance)

    if len(matches) < 200:
        frames.append(frames_arr[i])
print(len(frames))

In [ ]:
detected_faces = []
mp_face_detction = mp.solutions.face_detection
for frame in frames:
    with mp_face_detction.FaceDetection(model_selection = 1, min_detection_confidence = 0.5) as face_detection:
        results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        for detection in results.detections:
            box = detection.location_data.relative_bounding_box
            x_start, y_start = int(box.xmin * img.shape[1]), int(box.ymin * img.shape[0])
            x_end, y_end = int((box.xmin + box.width) * img.shape[1]), int((box.ymin + box.height) * img.shape[0])

            if x_start < 0 or y_start < 0:
                continue

            face = frame[y_start:y_end, x_start:x_end]
            face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            detected_faces.append(face)
print(len(detected_faces))

In [ ]:
%%time
from deepface import DeepFace as df
models=[]
res=[]
present=[]
for i in range(len(detected_faces)):
    model=df.find(img_path=detected_faces[i],db_path='Data/known_faces',model_name="Facenet",distance_metric="euclidean",enforce_detection=False,normalization="Facenet",detector_backend='retinaface')
    models.append(model)
print()
count = 0
for model in models:
    if len(model[0]) > 0:
        name=model[0]['identity'].values[0].split('\\')[-1].split('/')[-1].split('.')[-2]
        print(count , "_ ", name)
        if name not in present:
            present.append(name)
    else:
        print('Unknown Face detected')
    count += 1
print()
print(present)